In [11]:
from django.core.files.uploadedfile import SimpleUploadedFile
from django.test import RequestFactory
from django.contrib.auth.models import User
from django.views.generic import View
rf = RequestFactory()

In [8]:
f = open('static/dog.jpg', 'rb')
file_upload = SimpleUploadedFile('dog.jpg', f.read(),
                                 content_type='image/jpeg')

data = {
    'filename':file_upload, 'username' : 'song'
}
request = rf.post('/board/photo', data=data, format='multipart')

In [12]:
class PhotoView(View):
    def get(self, request):
        images = self.get_images(request.user.username)
        return render(request, 'board/photolist.html', {'images': images})

    def post(self, request):
        # 물리경로에 세이브
        file = request.FILES['filename']
        username = request.POST['username']
        save_path = f'{settings.BASE_DIR}/static/faces/{username}/{file.name}'
        fs = FileSystemStorage()
        fs.save(save_path, file)

        # db에 세이브
        cursor = connection.cursor()
        self.insert_img(username, file.name)
        return redirect('board:photo')

    def get_images(self, username):
        cursor = connection.cursor()
        cursor.execute(f'''
        SELECT filename FROM board_image i JOIN auth_user u
        ON i.author_id = u.id
        WHERE u.username = '{username}'
        ''')
        return dictfetchall(cursor)

    def insert_img(self, username, filename):
        cursor = connection.cursor()
        sql = f"SELECT id from auth_user WHERE username='{username}'"
        cursor.execute(sql)

        author_id = cursor.fetchone()[0]

        sql = f'''
        INSERT INTO board_image
        ("author_id", "filename")
        VALUES ({author_id}, '{filename}');
        '''
        cursor.execute(sql)

In [ ]:
if request.method == 'POST':
    post(request)